In [1]:
import os
%pwd
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    learning_rate: float
    subsample: float
    colsample_bytree: float
    random_state: int
    target_column: str

In [2]:
from fraud_detection_project.constants import *
from fraud_detection_project.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            learning_rate = params.learning_rate,
            subsample = params.subsample,
            colsample_bytree = params.colsample_bytree,
            random_state = params.random_state,
            target_column = schema.name
            
        )

        return model_trainer_config

In [6]:
import pandas as pd
import os
from fraud_detection_project import logger
from xgboost import XGBClassifier
import joblib

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        xgb_params = {
            'n_estimators':  self.config.n_estimators,
            'max_depth':  self.config.max_depth,
            'learning_rate':  self.config.learning_rate,
            'subsample':  self.config.subsample,
            'colsample_bytree': self.config.colsample_bytree,
            'random_state': self.config.random_state,
        }
        xgb_clf = XGBClassifier(**xgb_params)

        xgb_clf.fit(train_x, train_y)

        joblib.dump(xgb_clf, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-08-25 02:56:08,924: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-25 02:56:08,927: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-25 02:56:08,930: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-25 02:56:08,936: INFO: common: created directory at: artifacts]
[2023-08-25 02:56:08,939: INFO: common: created directory at: artifacts/model_trainer]
